In [1]:
import numpy as np
import pandas as pd
import ast as ast

In [2]:
movies = pd.read_csv('Exct/tmdb_5000_movies.csv')
credits = pd.read_csv('Exct/tmdb_5000_credits.csv') 

FileNotFoundError: [Errno 2] No such file or directory: 'Exct/tmdb_5000_movies.csv'

In [ ]:
movies.head()

In [ ]:
movies.head(1)

In [ ]:
movies = movies.merge(credits, on='title')

In [ ]:
movies.shape

In [ ]:
movies.head

In [ ]:
movies.head(1)

In [ ]:
movies.info()

In [ ]:
# Columns that are being kept
# Generes, id, keywords, title, overview, cast, crew

In [ ]:
movies = movies[['genres', 'id', 'keywords', 'title', 'overview', 'cast', 'crew']]

In [ ]:
movies.info()

In [ ]:
movies.head()

In [ ]:
movies.isnull().sum()

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.isnull().sum()

In [ ]:
movies.duplicated().sum()

In [ ]:
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [ ]:
movies['genres'] = movies['genres'].apply(convert)

In [ ]:
movies['genres']

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)

In [ ]:
def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            counter += 1
            L.append(i['name'])
        else:
            break
    return L

In [ ]:
movies['cast'] = movies['cast'].apply(convert3)
# movies['crew'] = movies['genres'].apply(convert3)

In [ ]:
movies.head()

In [ ]:
def fetch_job(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

In [ ]:
movies['crew'][0]

In [ ]:
movies['crew'] = movies['crew'].apply(fetch_job)

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [ ]:
movies['overview'].head()

In [ ]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ", "") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ", "") for i in x])

In [ ]:
movies.head()

In [ ]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [ ]:
movies.head()

In [ ]:
new_df = movies[['id', 'title', 'tags']]

In [ ]:
new_df

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
new_df.head()

In [ ]:
!pip install nltk

In [ ]:
import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(obj):
    y = []
    for i in obj.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)

In [ ]:
new_df['tags'][0]

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

In [ ]:
new_df['tags']

In [ ]:
new_df.head(1)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000, stop_words='english')

In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
sorted(list(enumerate(similarity[0])), reverse = True, key = lambda x:x[1])[1:6]

In [ ]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movie_list = sorted(list(enumerate(distances)), reverse = True, key = lambda x:x[1])[1:6]
    for i in movie_list:
        print(new_df.iloc[i[0]].title)
    

In [ ]:
recommend('Avatar')

In [ ]:
import pickle

In [ ]:
pickle.dump(new_df.to_dict(), open('movie_dict.pkl', 'wb'))

In [ ]:
pickle.dump(similarity, open('similarity.pkl', 'wb'))